# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from copy import deepcopy

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.non_reactive_stage import non_reactive_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Construct a reboiler

In [6]:
model.reboiler = pe.Block(rule=non_reactive_stage_rule)

> Importing Non Reactive Stage......
> Adding the following local variable:
------------------------------------
| reboiler.T_F
| reboiler.P
| reboiler.Q_main
| reboiler.x_
| reboiler.y_
| reboiler.x
| reboiler.y
| reboiler.z
| reboiler.L
| reboiler.V
| reboiler.F
| reboiler.H_L_
| reboiler.H_V_
| reboiler.H_L
| reboiler.H_V
| reboiler.T
| reboiler.H_F
| reboiler.f_V
| reboiler.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.energy_block.dH_F
| reboiler.energy_block.dH_V
| reboiler.energy_block.dH_L
| reboiler.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.VLE_block.P_VLE
| reboiler.VLE_block.n_ave
| reboiler.VLE_block.n_ave_cal
| reboiler.VLE_block.Hen
| reboiler.VLE_block.Hen0
| reboiler.VLE_block.gamma
| r

# Linking Stage Variables

### Vapor Between Reactive Stages

In [7]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [8]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [9]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [10]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

### Reboiler

In [11]:
def V_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].V['in'] == model.reboiler.V['out']
model.V_reboiler_con = pe.Constraint(rule=V_reboiler_rule)

def Vy_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].y_['in',i] == model.reboiler.y[i]
model.Vy_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_reboiler_rule)

def Vh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_V_['in'] == model.reboiler.H_V
model.Vh_reboiler_con = pe.Constraint(rule=Vh_reboiler_rule)

In [12]:
def L_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].L['out'] == model.reboiler.L['in']
model.L_reboiler_con = pe.Constraint(rule=L_reboiler_rule)

def Lx_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].x[i] == model.reboiler.x_['in',i]
model.Lx_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_reboiler_rule)

def Lh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_L == model.reboiler.H_L_['in']
model.Lh_reboiler_con = pe.Constraint(rule=Lh_reboiler_rule)

In [13]:
# model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load from single stage solutions

In [15]:
# with open('../saved_solutions/reactive_flash_MPCC_P_pf.pickle', 'rb') as f:
#     results_imported = pickle.load(f)
# results_changed = deepcopy(results_imported)

### Duplicate variable solution and bounds multiplier

In [16]:
# results_changed.Solution.Variable = {}
# for i in list(results_imported.Solution.Variable.keys()):
#     for j in model.TRAY:
#         results_changed.Solution.Variable['reactive[{}].{}'.format(j,i)] = \
#         results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [17]:
# results_changed.Solution.Constraint = {}
# for i in list(results_imported.Solution.Constraint.keys()):
#     for j in model.TRAY:
#         results_changed.Solution.Constraint['reactive[{}].{}'.format(j,i)] = \
#         results_imported.Solution.Constraint[i]

### Load changed solution into current model

In [18]:
# model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [19]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [20]:
# 'reboiler' fixing last stage V_in

# model.reactive[model.TRAY.last()].V['in'].fix(0)
# for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
# model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.F.fix(0)
model.reboiler.T_F.fix(300+273.15)
model.reboiler.z.fix(0)

model.reboiler.V['in'].fix(0)
for i in m.COMP_TOTAL: model.reboiler.y_['in',i].fix(0)
model.reboiler.H_V_['in'].fix(0)

# Load Operating Parameters

In [21]:
# condenser
model.condenser.P.fix(19)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2))
    model.reactive[j].z['H2'].fix(2/(1+2))
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(300+273.15)
    
# reboiler
model.reboiler.L['P'].fix(0)
model.reboiler.V['P'].fix(0)
model.reboiler.P.fix(20)
model.reboiler.T.setub(350+273.15)

In [22]:
check_DOF(pe,model)

Active Equality Constraints:	 3424
Active Inequality Constraints:	 0
Active Variables:		 3617
Fixed Variables:		 191
DOF:				 2


In [23]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
# opt.options['linear_solver'] = 'ma86'

# opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.11.1: print_user_options=yes
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                     no
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonze

  95r-5.2195810e+002 1.66e+001 2.40e+005  -0.9 8.39e-002   2.4 6.16e-001 7.92e-003h  5
  96r-5.2195810e+002 1.66e+001 3.81e+004  -0.9 4.93e-003   2.8 1.00e+000 1.00e+000f  1
  97r-5.2195205e+002 1.66e+001 1.02e+002  -0.9 9.67e-001    -  1.00e+000 1.00e+000f  1
  98r-5.2195205e+002 1.66e+001 2.03e+003  -2.4 3.30e-002   3.3 7.08e-001 6.59e-001f  1
  99r-5.2195205e+002 1.66e+001 8.87e+002  -2.4 3.26e-002   2.8 5.68e-001 4.95e-001f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100r-5.2195205e+002 1.66e+001 3.02e+002  -2.4 1.58e-002   2.3 5.83e-001 5.13e-001f  1
 101r-5.2195205e+002 1.66e+001 1.23e+002  -2.4 4.41e-002   1.8 6.39e-001 6.33e-001f  1
 102r-5.2195205e+002 1.66e+001 2.29e+003  -2.4 7.72e-002   1.3 7.36e-001 3.60e-001f  1
 103r-5.2195205e+002 1.66e+001 2.53e+000  -2.4 5.32e-003   2.7 1.00e+000 1.00e+000f  1
 104r-5.2195205e+002 1.66e+001 3.48e+001  -2.4 9.62e-003   2.2 1.00e+000 1.00e+000f  1
 105r-5.2195204e+002 1.66e+001 1.46e+001  -2.4 3.

 186 -5.2097348e+002 1.20e+001 2.76e+003  -1.0 9.62e+004    -  9.76e-005 8.36e-005f  1
 187r-5.2097348e+002 1.20e+001 9.99e+002   1.1 0.00e+000    -  0.00e+000 4.42e-007R  2
 188r-5.2097348e+002 1.20e+001 9.98e+002   1.1 1.20e+004    -  1.22e-003 1.41e-006f  2
 189r-5.2097576e+002 4.57e+000 9.96e+002   1.1 5.39e+003    -  1.39e-003 2.20e-003f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r-5.2097576e+002 4.57e+000 9.99e+002   0.7 0.00e+000    -  0.00e+000 2.52e-007R  5
 191r-5.2097576e+002 4.57e+000 9.99e+002   0.7 6.28e+002    -  5.18e-003 3.57e-009f  3
 192r-5.2098021e+002 4.52e+000 9.91e+002   0.7 3.58e+002    -  5.38e-003 7.80e-003f  1
 193r-5.2099039e+002 4.42e+000 2.77e+003   0.7 3.32e+002    -  5.96e-002 1.79e-002f  1
 194r-5.2100992e+002 3.98e+000 3.06e+003  -0.0 2.77e+002    -  1.57e-001 9.06e-002f  1
 195 -5.2099435e+002 3.98e+000 2.22e+002  -1.0 9.44e+004    -  1.52e-004 2.62e-005f  2
 196 -5.2099238e+002 3.98e+000 2.34e+002  -1.0 9.

 281r-5.1259835e+002 1.12e-001 6.11e+002  -5.6 9.24e-003  -1.7 4.25e-001 8.23e-001h  1
 282r-5.1259827e+002 1.12e-001 4.20e+002  -5.6 5.63e-002  -2.2 5.26e-001 3.99e-001h  1
 283r-5.1259823e+002 1.12e-001 3.47e+002  -5.6 1.71e-001  -2.7 4.54e-001 2.83e-001f  1
 284r-5.1259823e+002 1.12e-001 6.10e+002  -5.6 4.61e+000  -3.2 3.75e-002 1.93e-002h  1
 285r-5.1259822e+002 1.12e-001 1.21e+003  -5.6 1.30e+000  -3.6 2.11e-001 4.92e-002h  1
 286r-5.1259818e+002 1.12e-001 3.30e+002  -5.6 5.04e-001  -3.2 1.00e+000 4.82e-001f  1
 287r-5.1259800e+002 1.12e-001 6.01e+002  -5.6 5.79e-001    -  1.00e+000 1.14e-001h  1
 288r-5.1259641e+002 1.12e-001 4.14e+000  -5.6 4.15e-001    -  1.00e+000 1.00e+000f  1
 289r-5.1259720e+002 1.12e-001 1.46e-002  -5.6 1.19e-001    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290r-5.1259720e+002 1.12e-001 1.84e-006  -5.6 8.93e-004    -  1.00e+000 1.00e+000h  1
 291r-5.1259163e+002 1.12e-001 3.72e+001  -8.4 1.

 368r-5.1257462e+002 1.12e-001 9.86e+002  -8.4 1.31e-001   1.0 2.62e-001 3.95e-002f  1
 369r-5.1257462e+002 1.12e-001 1.10e+003  -8.4 1.19e+000   0.5 1.70e-002 4.00e-003f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r-5.1257462e+002 1.12e-001 1.49e+003  -8.4 1.38e-001   1.0 5.49e-001 2.80e-004h  1
 371r-5.1257462e+002 1.12e-001 1.38e+003  -8.4 1.20e+000   0.5 1.10e-002 2.05e-002f  1
 372r-5.1257462e+002 1.12e-001 1.11e+003  -8.4 1.36e-001   0.9 3.65e-001 1.40e-001f  1
 373r-5.1257462e+002 1.12e-001 1.21e+003  -8.4 4.64e-001   0.4 5.15e-002 3.10e-004f  1
 374r-5.1257462e+002 1.12e-001 1.16e+003  -8.4 5.46e+000  -0.1 6.28e-004 1.62e-003f  1
 375r-5.1257462e+002 1.12e-001 1.14e+003  -8.4 5.25e-001   0.4 9.68e-003 1.28e-002f  1
 376r-5.1257462e+002 1.12e-001 1.14e+003  -8.4 6.33e+000  -0.1 2.09e-003 7.81e-004f  1
 377r-5.1257462e+002 1.12e-001 1.30e+003  -8.4 5.94e-001   0.3 1.09e-001 8.55e-003f  1
 378r-5.1257462e+002 1.12e-001 1.51e+003  -8.4 1.

...

4518 -5.6858935e+002 2.61e-003 6.84e+001  -5.7 2.35e+001    -  2.22e-005 3.88e-004h 11
4519 -5.6858765e+002 2.59e-003 2.13e+002  -5.7 5.47e+001    -  3.93e-004 6.29e-003h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
4520 -5.6858682e+002 2.59e-003 2.63e+002  -5.7 5.34e+001    -  1.04e-002 3.12e-003h  8
4521 -5.6858600e+002 2.58e-003 3.12e+002  -5.7 5.30e+001    -  2.72e-002 3.11e-003h  8
4522 -5.6858520e+002 2.57e-003 3.61e+002  -5.7 5.21e+001    -  2.06e-002 3.10e-003h  8
4523 -5.6858442e+002 2.57e-003 4.09e+002  -5.7 5.14e+001    -  4.59e-002 3.09e-003h  8
4524 -5.6858332e+002 2.56e-003 4.52e+002  -5.7 7.15e+001    -  1.98e-002 3.09e-003h  8
4525 -5.6858258e+002 2.56e-003 5.03e+002  -5.7 4.93e+001    -  6.53e-001 3.06e-003h  8
4526 -5.6858242e+002 2.55e-003 5.63e+002  -5.7 2.39e+001    -  9.48e-002 3.02e-003h  8
4527 -5.6858235e+002 2.54e-003 6.30e+002  -5.7 2.50e+001    -  8.01e-001 3.00e-003h  8
4528 -5.6859146e+002 2.65e-003 7.20e+003  -5.7 1.

4609 -5.6042560e+002 1.07e-001 1.50e+005  -5.7 1.22e+004    -  3.26e-002 1.07e-004h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
4610 -5.6040480e+002 1.08e-001 1.49e+005  -5.7 1.22e+004    -  1.63e-004 3.31e-004h  8
4611 -5.6038413e+002 1.09e-001 2.91e+005  -5.7 1.22e+004    -  3.35e-002 3.29e-004h  8
4612 -5.6036360e+002 1.10e-001 2.93e+005  -5.7 1.22e+004    -  8.34e-004 3.26e-004h  8
4613 -5.6034321e+002 1.11e-001 4.36e+005  -5.7 1.22e+004    -  3.62e-002 3.24e-004h  8
4614 -5.6032296e+002 1.12e-001 4.40e+005  -5.7 1.22e+004    -  1.52e-003 3.21e-004h  8
4615 -5.6030285e+002 1.13e-001 5.92e+005  -5.7 1.22e+004    -  4.11e-002 3.19e-004h  8
4616 -5.6028287e+002 1.14e-001 5.98e+005  -5.7 1.22e+004    -  2.20e-003 3.17e-004h  8
4617 -5.6026303e+002 1.15e-001 7.62e+005  -5.7 1.23e+004    -  4.76e-002 3.14e-004h  8
4618 -5.6024332e+002 1.16e-001 7.71e+005  -5.7 1.23e+004    -  2.88e-003 3.12e-004h  8
4619 -5.6022375e+002 1.17e-001 9.34e+005  -5.7 1.

4698 -5.5944973e+002 1.35e-002 2.17e+005  -5.7 1.13e+004    -  2.65e-003 6.31e-005h 10
4699 -5.5944606e+002 1.36e-002 3.97e+005  -5.7 1.13e+004    -  4.25e-002 6.30e-005h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
4700 -5.5944238e+002 1.36e-002 4.07e+005  -5.7 1.13e+004    -  2.77e-003 6.29e-005h 10
4701 -5.5943872e+002 1.37e-002 5.85e+005  -5.7 1.13e+004    -  4.53e-002 6.27e-005h 10
4702 -5.5943506e+002 1.38e-002 5.95e+005  -5.7 1.13e+004    -  2.90e-003 6.26e-005h 10
4703 -5.5943140e+002 1.38e-002 7.77e+005  -5.7 1.13e+004    -  4.98e-002 6.25e-005h 10
4704 -5.5942776e+002 1.39e-002 7.88e+005  -5.7 1.13e+004    -  3.02e-003 6.23e-005h 10
4705 -5.5942412e+002 1.39e-002 9.82e+005  -5.7 1.13e+004    -  5.60e-002 6.22e-005h 10
4706 -5.5942049e+002 1.40e-002 9.93e+005  -5.7 1.13e+004    -  3.14e-003 6.21e-005h 10
4707 -5.5756372e+002 1.61e+001 3.63e+006  -5.7 1.13e+004    -  6.03e-002 3.17e-002w  1
4708 -5.5756729e+002 1.83e+001 3.61e+006  -5.7 6.

4789 -5.6461599e+002 1.29e+001 4.06e+010  -5.7 1.67e-003  15.6 6.74e-001 6.00e-003h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
4790 -5.6461701e+002 1.29e+001 4.06e+010  -5.7 7.66e+002    -  2.13e-011 2.68e-004f  1
4791 -5.6461720e+002 1.20e-001 1.91e+009  -5.7 1.50e+001    -  2.14e-008 1.00e+000h  1
4792 -5.6461720e+002 1.19e-001 1.91e+009  -5.7 2.73e-006  15.1 1.04e-002 7.69e-003h  1
4793r-5.6461720e+002 1.19e-001 9.99e+002  -0.9 0.00e+000  14.7 0.00e+000 4.97e-009R  2
4794r-5.6461834e+002 1.17e-001 9.99e+002  -0.9 1.38e+004    -  1.33e-005 1.63e-005f  1
4795r-5.6461947e+002 1.12e-001 9.99e+002  -0.9 5.47e+003    -  1.81e-004 4.09e-005f  1
4796r-5.6462076e+002 9.77e-002 2.09e+003  -0.9 1.70e+003    -  7.94e-004 1.50e-004f  1
4797 -5.6658953e+002 1.86e+000 1.02e+003  -5.7 4.69e+005    -  6.14e-003 8.32e-004f  1
4798 -5.6659321e+002 3.99e+000 1.59e+005  -5.7 6.92e+002    -  4.79e-001 9.96e-001f  1
4799 -5.6659428e+002 3.99e+000 1.59e+005  -5.7 1.

4881 -5.7315001e+002 1.79e-010 1.38e-009  -8.6 7.02e-004    -  1.00e+000 1.00e+000h  1

Number of Iterations....: 4881

                                   (scaled)                 (unscaled)
Objective...............: -5.7315000572899407e+002  -5.7315000572899407e+002
Dual infeasibility......:  1.3768108458878103e-009   1.3768108458878103e-009
Constraint violation....:  1.7901861416869114e-010   1.7901861416869114e-010
Complementarity.........:  2.5059035604722630e-009   2.5059035604722630e-009
Overall NLP error.......:  2.5059035604722630e-009   2.5059035604722630e-009


Number of objective function evaluations             = 30109
Number of objective gradient evaluations             = 3924
Number of equality constraint evaluations            = 30110
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 5105
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 4881
Total 

In [25]:
model.reboiler.T.value

599.2725429430054

In [27]:
model.reboiler.L['out'].value

8.203503971277333e-08

In [24]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -17.3943689303 	         0.1850891983 	         0.0000000000
              300.00 	       -47.0548778403 	         0.4568201240 	         0.0000002138
----------------------------------------------------------------------------------------------------
Top
V	 0.18508919825496953
L	 0.010598397189567317
W	 0.2611325285073373
----------------------------------------------------------------------------------------------------
Bottom
L	 2.138313034897948e-07
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Last Stage	Vapor		Liquid
H2 		0.526%		0.007%		 H2       	0.213%		0.070%
CO 		0.008%		0.000%		 CO       	0.003%		0.001%
CO2 		19.175%		3.726%		 CO2      	7.856%		5.354%
H2O 		0.223%		0.000%		 H2O      	57.254%		59.714%
C2H4 		13.612%		4.251%		 C

In [26]:
model.reactive[1].kinetics_block.r_FT_total.value

0.2974317650215887

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/3_stage_condenser_240C.pickle','wb') as f:
#     pickle.dump(results,f)

# So, what exactly does adding a reflux do?

In [30]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [31]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [32]:
Refluxrange = np.linspace(0,2,21)
Trange = np.linspace(300+273.15,200+273.15,21)

In [41]:
cd_data_master = {}
rf_data_master = {}

for t in Trange:
    model.del_component(model.obj)
#     model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
    model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)
    model.solutions.load_from(results_anchor)
    update_dual(pe,model)
    
    for j in model.reactive:
        model.reactive[j].T.fixed = False
        model.reactive[j].T.setub(t)
        model.reactive[j].T = t
    model.condenser.L['out'].fix(0)
    
    results_anchor = opt.solve(model,tee=False)
    model.solutions.store_to(results_anchor)
    
    model.del_component(model.obj)
#     model.obj = pe.Objective(expr = model.condenser.L['out'] - sum(model.reactive[j].MPCC.pf for j in model.TRAY),sense=pe.maximize)
    model.obj = pe.Objective(expr = model.condenser.L['out'],sense=pe.maximize)
    model.condenser.L['out'].fixed = False
    for j in model.reactive:
        model.reactive[j].T.fixed=True
    
    print('\nWorking on T = {} K'.format(model.reactive[1].T.value))
    print('-'*108)
    
    cd_data = {};
    cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
    cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {}; cd_data['b'] = {}
    for i in m.COMP_TOTAL:
        cd_data['x'][i] = []
        cd_data['y'][i] = []
        cd_data['g'][i] = []
        cd_data['d'][i] = []
        cd_data['b'][i] = []

    rf_data = {}
    for j in model.reactive:
        rf_data[j] = {}
        rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
        rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
        rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []; rf_data[j]['pf'] = []
        for i in m.COMP_TOTAL:
            rf_data[j]['r'][i] = []
            rf_data[j]['b'][i] = []
            rf_data[j]['x'][i] = []
            rf_data[j]['y'][i] = []       

    for re in Refluxrange:
        model.condenser.L['out'].setub(re)
        results = opt.solve(model,tee=False)
        update_dual(pe,model)
        print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
              format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reboiler.L['out'].value))

        cd_data['V'].append(model.condenser.V['out'].value)
        cd_data['D'].append(model.condenser.L['P'].value)
        cd_data['Re'].append(model.condenser.L['out'].value)

        for i in model.reactive[1].r_total_comp:
            cd_data['x'][i].append(model.condenser.x[i].value)
            cd_data['y'][i].append(model.condenser.y[i].value)
            cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
            cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
            cd_data['b'][i].append(model.condenser.x[i].value*model.condenser.L['out'].value)

        for j in model.reactive:      
            rf_data[j]['T'].append(model.reactive[j].T.value)
            rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
            rf_data[j]['V'].append(model.reactive[j].V['out'].value)
            rf_data[j]['L'].append(model.reactive[j].L['out'].value)
            rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
            rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)
#             rf_data[j]['pf'].append(model.reactive[j].MPCC.pf.value)

            for i in model.reactive[1].r_total_comp:
                rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
                rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
                rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
                rf_data[j]['y'][i].append(model.reactive[j].y[i].value)
        
    cd_data_master[t] = cd_data
    rf_data_master[t] = rf_data


Working on T = 573.15 K
------------------------------------------------------------------------------------------------------------
Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.185 kmol/s	|Distillate = 0.011 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.100 kmol/s	|Vapor = 0.186 kmol/s	|Distillate = 0.011 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.200 kmol/s	|Vapor = 0.187 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.300 kmol/s	|Vapor = 0.187 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.400 kmol/s	|Vapor = 0.188 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.188 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.189 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.700 kmol/s	|Vapor = 0.189 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.800 kmol/s	|Vapor = 0.190 kmol/

Solved	|Reflux = 1.300 kmol/s	|Vapor = 0.164 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.400 kmol/s	|Vapor = 0.165 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.500 kmol/s	|Vapor = 0.166 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.600 kmol/s	|Vapor = 0.166 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.700 kmol/s	|Vapor = 0.167 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.800 kmol/s	|Vapor = 0.168 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.900 kmol/s	|Vapor = 0.168 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 2.000 kmol/s	|Vapor = 0.169 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.000 kmol/s

Working on T = 553.15 K
------------------------------------------------------------------------------------------------------------
Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.136 kmol/

Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.144 kmol/s	|Distillate = 0.031 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.148 kmol/s	|Distillate = 0.030 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.700 kmol/s	|Vapor = 0.151 kmol/s	|Distillate = 0.030 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.800 kmol/s	|Vapor = 0.153 kmol/s	|Distillate = 0.029 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.900 kmol/s	|Vapor = 0.155 kmol/s	|Distillate = 0.029 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 1.000 kmol/s	|Vapor = 0.157 kmol/s	|Distillate = 0.028 kmol/s	|Bottom = 0.001 kmol/s
Solved	|Reflux = 1.100 kmol/s	|Vapor = 0.159 kmol/s	|Distillate = 0.028 kmol/s	|Bottom = 0.001 kmol/s
Solved	|Reflux = 1.200 kmol/s	|Vapor = 0.161 kmol/s	|Distillate = 0.028 kmol/s	|Bottom = 0.001 kmol/s
Solved	|Reflux = 1.300 kmol/s	|Vapor = 0.163 kmol/s	|Distillate = 0.027 kmol/s	|Bottom = 0.001 kmol/s
Solved	|Reflux = 1.400 kmol/s	|Vapor = 0.164 kmol/s	|Distillate = 0.027 kmol/s	|Bo

Solved	|Reflux = 1.900 kmol/s	|Vapor = 0.190 kmol/s	|Distillate = 0.015 kmol/s	|Bottom = 0.010 kmol/s
Solved	|Reflux = 2.000 kmol/s	|Vapor = 0.193 kmol/s	|Distillate = 0.014 kmol/s	|Bottom = 0.010 kmol/s

Working on T = 518.15 K
------------------------------------------------------------------------------------------------------------
Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.124 kmol/s	|Distillate = 0.034 kmol/s	|Bottom = 0.000 kmol/s
Solved	|Reflux = 0.100 kmol/s	|Vapor = 0.132 kmol/s	|Distillate = 0.030 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.200 kmol/s	|Vapor = 0.139 kmol/s	|Distillate = 0.026 kmol/s	|Bottom = 0.005 kmol/s
Solved	|Reflux = 0.300 kmol/s	|Vapor = 0.145 kmol/s	|Distillate = 0.023 kmol/s	|Bottom = 0.007 kmol/s
Solved	|Reflux = 0.400 kmol/s	|Vapor = 0.151 kmol/s	|Distillate = 0.021 kmol/s	|Bottom = 0.008 kmol/s
Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.156 kmol/s	|Distillate = 0.019 kmol/s	|Bottom = 0.009 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.161 kmol/

Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.936 kmol/s	|Vapor = 0.215 kmol/s	|Distillate = 0.000 kmol/s	|Bo

Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.069 kmol/s	|Vapor = 0.217 kmol/s	|Distillate = 0.000 kmol/s	|Bo

In [42]:
cnumber_range = range(1,57)

In [43]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [44]:
g_data = {}
d_data = {}
lr_data = {}
l1_data = {}
l2_data = {}
l3_data = {}

cd_x_data = {}
rf_x_data = {}

for t in Trange:

    g_data[t] = trans_cnumber(cd_data_master[t]['g'])
    d_data[t] = trans_cnumber(cd_data_master[t]['d'])
    lr_data[t] = trans_cnumber(cd_data_master[t]['d'])
    l1_data[t] = trans_cnumber(rf_data_master[t][1]['b'])
    l2_data[t] = trans_cnumber(rf_data_master[t][2]['b'])
    l3_data[t] = trans_cnumber(rf_data_master[t][3]['b'])
    
    cd_x_data[t] = trans_cnumber(cd_data_master[t]['x'])
    rf_x_data[t] = {}
    for j in model.reactive:
        rf_x_data[t][j] = trans_cnumber(rf_data_master[t][j]['x'])

KeyError: 2

In [45]:
 def plot_distribution(index,temperature):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[temperature][index],'co-')
    ax.plot(cnumber_range,d_data[temperature][index],'go-')
    ax.plot(cnumber_range,lr_data[temperature][index],'ro-',alpha=0.1)
    ax.plot(cnumber_range,l1_data[temperature][index],'ro-',alpha=0.3)
    ax.plot(cnumber_range,l2_data[temperature][index],'ro-',alpha=0.5)
    ax.plot(cnumber_range,l3_data[temperature][index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Reflux','L1','L2','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data_master[temperature]['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[temperature][index],'go-')
    ax2.plot(cnumber_range,rf_x_data[temperature][1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[temperature][2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[temperature][3][index],'ro-')

    ax2.set_ylim(0, 0.2)
    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [46]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:'),temperature=widgets.SelectionSlider(
    options=Trange,
    value=Trange[0],
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)
);

A Jupyter Widget

In [47]:
plot_distribution(20,523.15)

KeyError: 523.15